In [67]:
# 遷移學習，即是採用別人已經訓練好的模型, 只需調整最後或最後幾個Layer
# https://pytorch.org/vision/stable/models.html#classification
import requests
import zipfile
import torch
from torch.utils.data import Dataset
from pathlib import Path
from PIL import Image

In [68]:
url = "https://firebasestorage.googleapis.com/v0/b/grandmacan-2dae4.appspot.com/o/ML_data%2Fone_piece_full.zip?alt=media&token=937656fd-f5c1-44f5-b174-1e2d590b8ef3"

with open("one_piece_full.zip", "wb") as f:
  req = requests.get(url)
  f.write(req.content)

with zipfile.ZipFile("one_piece_full.zip", "r") as zip_file:
  zip_file.extractall("one_piece_full")

In [69]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [70]:
# 貼前面寫過的來用
def accuracy_fn(y_pred, y_true):
  correct_num = (y_pred==y_true).sum()
  acc = correct_num / len(y_true) * 100
  return acc

def train_step(dataloader, model, cost_fn, optimizer, accuracy_fn, device):
  train_cost = 0
  train_acc = 0
  for batch, (x, y) in enumerate(dataloader):
    x = x.to(device)
    y = y.to(device)

    model.train()

    y_pred = model(x)

    cost = cost_fn(y_pred, y)

    train_cost += cost
    train_acc += accuracy_fn(y_pred.argmax(dim=1), y)

    optimizer.zero_grad()

    cost.backward()

    optimizer.step()

  train_cost /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  print(f"\nTrain Cost: {train_cost:.4f}, Train Acc: {train_acc:.2f}")


def test_step(dataloader, model, cost_fn, accuracy_fn, device):
  test_cost = 0
  test_acc = 0
  model.eval()
  with torch.inference_mode():
    for x, y in dataloader:
      x = x.to(device)
      y = y.to(device)

      test_pred = model(x)

      test_cost += cost_fn(test_pred, y)
      test_acc += accuracy_fn(test_pred.argmax(dim=1), y)

    test_cost /= len(test_dataloader)
    test_acc /= len(test_dataloader)

  print(f"Test Cost: {test_cost:.4f}, Test Acc: {test_acc:.2f} \n")

In [71]:
# 貼前面寫過的來用
class ImageDataset(Dataset):
  def __init__(self, root, train, transform=None):

    if train:
      image_root = Path(root) / "train"
    else:
      image_root = Path(root) / "test"

    with open(Path(root) / "classnames.txt", "r") as f:
      lines = f.readlines()
      self.classes = [line.strip() for line in lines]

    self.paths = [i for i in image_root.rglob("*") if i.is_file()]
    self.transform = transform

  def __getitem__(self, index):
    img = Image.open(self.paths[index]).convert("RGB")
    class_name = self.paths[index].parent.name
    class_idx = self.classes.index(class_name)

    if self.transform:
      return self.transform(img), class_idx
    else:
      return img, class_idx


  def __len__(self):
    return len(self.paths)

In [72]:
import torchvision
weights = torchvision.models.EfficientNet_B1_Weights.DEFAULT # 參數
model = torchvision.models.efficientnet_b1(weights=weights) # 模型
# model

In [73]:
# 了解參數在此模型中經過了哪些轉換
weights.transforms()

ImageClassification(
    crop_size=[240]
    resize_size=[255]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [74]:
# 讓我們的資料集在訓練前也先做一樣的轉換
efficientnet_b1_transforms = weights.transforms()

In [75]:
# 貼前面寫過的來用
# 創建訓練集與測試集Dataset
train_dataset = ImageDataset(root="one_piece_full", train=True, transform=efficientnet_b1_transforms)
test_dataset = ImageDataset(root="one_piece_full", train=False, transform=efficientnet_b1_transforms)

In [76]:
# 貼前面寫過的來用
# 創建Dataloader 將資料分成數個Batch
from torch.utils.data import DataLoader

BATCH_SIZE = 16

train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [77]:
len(train_dataloader), len(test_dataloader)

(189, 47)

In [78]:
!pip install torchinfo
from torchinfo import summary

In [79]:
# 用上面引入的torchinfo 看模型架構
summary(model=model,
    input_size=(16, 3, 64, 64),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape               Output Shape              Param #                   Trainable
EfficientNet (EfficientNet)                                  [16, 3, 64, 64]           [16, 1000]                --                        True
├─Sequential (features)                                      [16, 3, 64, 64]           [16, 1280, 2, 2]          --                        True
│    └─Conv2dNormActivation (0)                              [16, 3, 64, 64]           [16, 32, 32, 32]          --                        True
│    │    └─Conv2d (0)                                       [16, 3, 64, 64]           [16, 32, 32, 32]          864                       True
│    │    └─BatchNorm2d (1)                                  [16, 32, 32, 32]          [16, 32, 32, 32]          64                        True
│    │    └─SiLU (2)                                         [16, 32, 32, 32]          [16, 32, 32, 32]          --                

In [80]:
# 取得classifier 層
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [81]:
# 取得最後那個Linear 層, 因為我們要將最終的輸出Layer 從原本1000 改成 18
from torch import nn
model.classifier[1] = nn.Linear(in_features=1280, out_features=18, bias=True)

In [82]:
# 沒有追蹤梯度(定義為False)就不會再重新訓練參數, 我們僅需訓練最後classifier 部分, 因此將前面features 的梯度追蹤(requires_grad)都設定為False
for param in model.features.parameters():
  param.requires_grad=False
  # print(param)

In [83]:
# 貼前面寫過的來用
cost_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
# 貼前面寫過的來用
from tqdm.auto import tqdm

epochs = 10

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-------")

  train_step(train_dataloader, model, cost_fn, optimizer, accuracy_fn, device)

  test_step(test_dataloader, model, cost_fn, accuracy_fn, device)


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
-------

Train Cost: 2.3300, Train Acc: 51.40
Test Cost: 1.9059, Test Acc: 66.13 

Epoch: 1
-------

Train Cost: 1.5813, Train Acc: 72.28
Test Cost: 1.4717, Test Acc: 71.87 

Epoch: 2
-------

Train Cost: 1.2351, Train Acc: 76.23
Test Cost: 1.2423, Test Acc: 75.08 

Epoch: 3
-------

Train Cost: 1.0255, Train Acc: 79.30
Test Cost: 1.0816, Test Acc: 76.94 

Epoch: 4
-------

Train Cost: 0.9036, Train Acc: 81.16
Test Cost: 0.9983, Test Acc: 78.40 

Epoch: 5
-------

Train Cost: 0.8101, Train Acc: 83.46
Test Cost: 0.9107, Test Acc: 79.75 

Epoch: 6
-------

Train Cost: 0.7202, Train Acc: 84.93
Test Cost: 0.8411, Test Acc: 81.10 

Epoch: 7
-------

Train Cost: 0.6725, Train Acc: 85.70
Test Cost: 0.8324, Test Acc: 81.50 

Epoch: 8
-------

Train Cost: 0.6276, Train Acc: 86.39
Test Cost: 0.8004, Test Acc: 81.21 

Epoch: 9
-------

Train Cost: 0.5852, Train Acc: 87.43
Test Cost: 0.7800, Test Acc: 81.23 



In [ ]:
# 貼前面寫過的來用
# 引入圖片"luffy.jpeg" 並做分類預測
img = Image.open("luffy.jpeg").convert("RGB")
img = efficientnet_b1_transforms(img)
# img.shape 輸出為(3, 224, 224)
img = img.reshape(-1, 3, 224, 224) # 變四維
model.eval()
with torch.inference_mode():
  y_pred = model(img.to(device))

y_pred = torch.softmax(y_pred, dim=1)
class_idx = y_pred.argmax(dim=1)
train_dataset.classes[class_idx]